In [15]:
# Parameters
name = "Gülnarə"
surname = "Əzizova"
sector = "S\u0259hhiyy\u0259"
field = "Infeksiya v\u0259 Parazit X\u0259st\u0259likl\u0259ri"
start_year = 2024
end_year = 2026

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
import matplotlib
matplotlib.use('Agg')
warnings.filterwarnings('ignore')

In [17]:
# Prophet model (alternative implementation)
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except ImportError:
    PROPHET_AVAILABLE = False
    print("Prophet not available, using alternative forecasting methods")

In [18]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as patches
from datetime import datetime, timedelta

In [19]:
# Set matplotlib to use available fonts
plt.rcParams['font.family'] = ['DejaVu Sans', 'sans-serif']
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (12, 8)
# Enable Unicode support
plt.rcParams['axes.unicode_minus'] = False

# Infeksiyon və Parazit Xəstəlikləri üzrə Statistik Göstəricilər

In [20]:
# Read the CSV data
def read_infection_and_parasite_data():
  df2 = pd.read_csv('infeksion_və_parazit_xəstəlikləri.csv')
  df2['Illər'] = pd.to_datetime(df2['Illər'], format='%Y')
  df2.set_index('Illər', inplace=True)
  return df2

read_infection_and_parasite_data()

,Bruselyoz,Difteriya,Epidemik parotit,Göyöskürək,Məxmərək,Pedikulyoz,Qarayara,"Qarın yatalağı və paratif A,B,C",Qoturluq,Qrip və yuxarı tənəffüs yollarının kəskin infeksiyası,Qrip və yuxarı tənəffüs yollarının kəskin infeksiyası,Qızılça,Salmonelyoz infeksiyaları,Skarlatin,Suçiçəyi,Tetanus,Viruslu hepatitlər,Ümumi kəskin bağırsaq infeksiyaları,İlk dəfə diaqnoz qoyulmuş malyariya
Illər,,,,,,,,,,,,,,,,,,,
2000-01-01,649.0,1.012705,3473.070784,5.569876,110.384813,1461.332889,1.012705,7.595285,1436.015271,4162.010392,327596.0,107.346699,279.000000,62.281339,1034.984208,1.519057,1205.624951,3709.560903,774.719099
2005-01-01,415.0,NaN,153.307811,1.011933,518.615533,576.295699,2.529832,5.565630,639.035530,4201.245779,352077.0,626.386370,288.400833,26.310251,2611.798420,NaN,855.589138,5322.766249,122.949829
2007-01-01,475.0,2.023248,65.249738,6.069743,2.023248,338.893990,8.598803,2.529060,350.021853,4316.389632,371339.0,NaN,213.452633,21.749913,3183.074451,1.011624,1124.419912,6960.983727,53.616064
2008-01-01,405.0,1.517205,41.470264,2.528675,NaN,219.488960,6.574554,0.505735,363.623416,4453.346333,388265.0,2.528675,207.351322,40.964529,2519.571429,3.034410,1195.557378,6457.729340,35.401445
2009-01-01,392.0,1.011328,117.314105,2.528321,NaN,177.488150,NaN,1.516993,314.523161,4459.257077,393632.0,NaN,180.016472,93.547886,3850.633234,0.505664,941.041162,5934.981257,40.453140
2010-01-01,364.0,NaN,63.199551,7.078350,0.505596,147.634152,0.505596,3.539175,258.359765,3842.808054,343328.0,NaN,152.184519,72.805883,3520.467798,1.516789,623.400373,6851.336943,26.291013
2011-01-01,426.0,NaN,50.552297,12.638074,NaN,104.643255,1.011046,NaN,233.551612,4002.297556,362332.0,NaN,127.400000,117.786852,5281.703991,3.033138,394.307917,6509.113762,4.044184
2012-01-01,289.0,NaN,64.192045,9.098085,NaN,97.551691,7.581738,NaN,215.321349,4149.916084,380788.0,NaN,205.212366,101.089835,4420.153050,3.538144,336.123703,7037.368888,1.516348
2013-01-01,250.0,NaN,39.924877,2.021513,NaN,68.226056,5.559160,NaN,169.807073,4305.567507,400280.0,51.548576,340.119525,95.011101,5003.244127,2.526891,264.818174,6810.476551,NaN


In [21]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [22]:
def linear_trend_forecast_2(series, periods=3):
    """Linear trend forecasting as fallback"""
    X = np.arange(len(series)).reshape(-1, 1)
    y = series.values

    model = LinearRegression()
    model.fit(X, y)

    future_X = np.arange(len(series), len(series) + periods).reshape(-1, 1)
    forecast = model.predict(future_X)

    # Simple confidence interval estimation
    residuals = y - model.predict(X)
    std_error = np.std(residuals)
    conf_int = np.column_stack([forecast - 1.96*std_error, forecast + 1.96*std_error])

    return forecast, conf_int, model

In [23]:
def arima_forecast_2(series, periods=3, order=(1,1,1)):
    """ARIMA forecasting"""
    try:
        model = ARIMA(series, order=order)
        fitted_model = model.fit()
        forecast = fitted_model.forecast(steps=periods)
        conf_int = fitted_model.get_forecast(steps=periods).conf_int()
        return forecast.values, conf_int.values, fitted_model
    except:
        try:
            # Fallback to simple ARIMA
            model = ARIMA(series, order=(1,1,0))
            fitted_model = model.fit()
            forecast = fitted_model.forecast(steps=periods)
            conf_int = fitted_model.get_forecast(steps=periods).conf_int()
            return forecast.values, conf_int.values, fitted_model
        except:
            # Final fallback to linear trend
            return linear_trend_forecast_2(series, periods)

In [24]:
def prophet_forecast_2(series, periods=3):
    """Prophet forecasting (if available)"""
    if not PROPHET_AVAILABLE:
        return linear_trend_forecast(series, periods)

    try:
        df = pd.DataFrame({
            'ds': series.index,
            'y': series.values
        })

        model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
        model.fit(df)

        future = model.make_future_dataframe(periods=periods, freq='Y')
        forecast = model.predict(future)

        forecast_values = forecast.tail(periods)['yhat'].values
        conf_int = forecast.tail(periods)[['yhat_lower', 'yhat_upper']].values

        return forecast_values, conf_int, model
    except:
        return linear_trend_forecast_2(series, periods)

In [25]:
def random_forest_forecast_2(series, periods=3):
    """Random Forest forecasting"""
    # Create features (lagged values)
    n_lags = min(5, len(series) // 2)
    X, y = [], []

    for i in range(n_lags, len(series)):
        X.append(series.values[i-n_lags:i])
        y.append(series.values[i])

    X, y = np.array(X), np.array(y)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)

    # Generate forecasts
    forecasts = []
    last_values = series.values[-n_lags:]

    for _ in range(periods):
        pred = model.predict([last_values])[0]
        forecasts.append(pred)
        last_values = np.append(last_values[1:], pred)

    return np.array(forecasts), None, model

In [26]:
def perform_infection_and_parasite_analysis():
    df = read_infection_and_parasite_data()

    # Select key diseases to analyze (focusing on those with complete data)
    diseases = [
        'Brusel oz',
        'Göyöskürək',
        'Məxmərək',
        'Pedikulyoz',
        'Qarayara',
        'Qrip və yuxarı tənəffüs yollarının kəskin infeksiyası',
        'Qızılça',
        'Suçiçəyi',
        'Viruslu hepatitlər',
        'Ümumi kəskin bağırsaq infeksiyaları'
    ]

    # Filter for diseases that have sufficient data
    diseases = [d for d in diseases if d in df.columns and df[d].count() > 5]

    results = {}

    for disease in diseases:
        series = df[disease].dropna()

        if len(series) < 3:
            continue  # Skip diseases with insufficient data

        # ARIMA forecast
        arima_pred, arima_conf, arima_model = arima_forecast_2(series)

        # Prophet forecast
        prophet_pred, prophet_conf, prophet_model = prophet_forecast_2(series)

        # Random Forest forecast
        rf_pred, rf_conf, rf_model = random_forest_forecast_2(series)

        # Linear trend forecast
        linear_pred, linear_conf, linear_model = linear_trend_forecast_2(series)

        results[disease] = {
            'historical': series,
            'arima': {'forecast': arima_pred, 'conf_int': arima_conf, 'model': arima_model},
            'prophet': {'forecast': prophet_pred, 'conf_int': prophet_conf, 'model': prophet_model},
            'rf': {'forecast': rf_pred, 'conf_int': rf_conf, 'model': rf_model},
            'linear': {'forecast': linear_pred, 'conf_int': linear_conf, 'model': linear_model}
        }

    return results

In [27]:
def create_forecast_plot_2(disease, data, ax):
    """Create forecast visualization for a disease with custom methods for each"""
    historical = data['historical']

    # Plot historical data
    ax.plot(historical.index, historical.values, 'o-', label='Tarixi məlumatlar',
            color='blue', linewidth=2, markersize=4)

    # Future years
    future_years = pd.date_range(start=historical.index[-1], periods=4, freq='Y')[1:]

    # Define which methods to keep for each disease
    methods_config = {
        'Bruselyoz': ['arima', 'linear'],
        'Göyöskürək': ['arima', 'rf'],
        'Məxmərək': ['arima', 'prophet'],
        'Pedikulyoz': ['arima', 'linear'],
        'Qarayara': ['arima', 'rf'],
        'Qrip və yuxarı tənəffüs yollarının kəskin infeksiyası': ['arima', 'prophet', 'linear'],
        'Qızılça': ['arima', 'linear'],
        'Suçiçəyi': ['arima', 'rf', 'prophet'],
        'Viruslu hepatitlər': ['arima', 'linear'],
        'Ümumi kəskin bağırsaq infeksiyaları': ['arima', 'prophet']
    }

    # Method display names and colors
    method_info = {
        'arima': {'name': 'ARIMA', 'color': 'red'},
        'prophet': {'name': 'Prophet', 'color': 'green'},
        'rf': {'name': 'Random Forest', 'color': 'orange'},
        'linear': {'name': 'Xətti Trend', 'color': 'purple'}
    }

    # Get methods to plot for this disease
    methods_to_plot = methods_config.get(disease, ['arima', 'prophet'])

    # Plot forecasts for selected methods
    for method in methods_to_plot:
        info = method_info[method]
        forecast = data[method]['forecast']

        ax.plot(future_years, forecast, 'o--',
                label=f'{info["name"]} proqnozu',
                color=info['color'], linewidth=2, markersize=6)

        # Add confidence intervals if available
        conf_int = data[method]['conf_int']
        if conf_int is not None:
            try:
                if hasattr(conf_int, 'values'):
                    conf_int = conf_int.values
                if conf_int.ndim == 2 and conf_int.shape[1] >= 2:
                    ax.fill_between(future_years, conf_int[:, 0], conf_int[:, 1],
                                  alpha=0.2, color=info['color'])
                elif conf_int.ndim == 1 and len(conf_int) >= 2:
                    margin = np.std(forecast) * 0.5
                    ax.fill_between(future_years, forecast - margin, forecast + margin,
                                  alpha=0.2, color=info['color'])
            except:
                pass  # Skip if error

    ax.set_title(f'{disease} - Zaman Seriyası Analizi və Proqnoz',
                fontsize=14, fontweight='bold')
    ax.set_xlabel('İl', fontsize=12)
    ax.set_ylabel('Xəstəlik hallarının sayı', fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

    # Add trend line
    from scipy import stats
    years_numeric = np.arange(len(historical))
    slope, intercept, r_value, p_value, std_err = stats.linregress(years_numeric, historical.values)

    # Extend trend line to future
    all_years_numeric = np.arange(len(historical) + 3)
    trend_line = slope * all_years_numeric + intercept
    all_years = list(historical.index) + list(future_years)

    # Disease-specific explanations
    disease_explanations = {
        'Bruselyoz': (
            "Bruselyoz xəstəliyinin halları son illərdə azalma tendensiyası göstərir. "
            "Bu azalma əsasən heyvandarlıq sahəsində sanitariya tədbirlərinin yaxşılaşdırılması "
            "və profilaktik tədbirlərin artırılması ilə əlaqədardır. Proqnozlar göstərir ki, "
            "bu azalma tendensiyası davam edəcək."),

        'Göyöskürək': (
            "Göyöskürək xəstəliyinin halları son illərdə dalğalanma göstərir. "
            "Bu xəstəliyin təbii dövriliyi və peyvənd proqramlarının effektivliyi "
            "ilə əlaqədardır. Proqnozlar göstərir ki, yaxın illərdə halların sayı "
            "sabit qalacaq və ya bir qədər azalacaq."),

        'Qrip və yuxarı tənəffüs yollarının kəskin infeksiyası': (
            "Qrip və yuxarı tənəffüs yolları infeksiyaları ən çox yayılmış xəstəliklərdəndir. "
            "Həllərin sayı illər ərzində dalğalanma göstərir ki, bu da qrip virusunun "
            "mütənasib dəyişməsi ilə əlaqədardır. Proqnozlar göstərir ki, yaxın illərdə "
            "bu xəstəliklərin hallarının sayı sabit qalacaq."),

        'Viruslu hepatitlər': (
            "Viruslu hepatitlər xəstəliyinin halları son illərdə azalma tendensiyası göstərir. "
            "Bu, peyvənd proqramlarının uğuru və sanitariya şəraitinin yaxşılaşması ilə "
            "əlaqədardır. Proqnozlar göstərir ki, bu azalma tendensiyası davam edəcək."),

        'Ümumi kəskin bağırsaq infeksiyaları': (
            "Bağırsaq infeksiyalarının halları illər ərzində dalğalanma göstərir. "
            "Bu, əsasən yay aylarında artır və sanitariya şəraitindən asılıdır. "
            "Proqnozlar göstərir ki, yaxın illərdə halların sayı sabit qalacaq.")
    }

    # Format these texts as paragraphs
    import textwrap
    explanations = disease_explanations.get(disease, "")
    wrapped_texts = textwrap.fill(explanations, width=100)

    ax.plot(all_years, trend_line, '--', color='gray', alpha=0.7,
            label=f'Trend (R²={r_value**2:.3f})')
    ax.legend(fontsize=9)

    # Add explanation text below the plot
    ax.text(0.5, -0.25, wrapped_texts,
            ha='center', va='top', transform=ax.transAxes,
            fontsize=10, style='italic', color='#555555')

    # Add values for each corresponding years
    for year, value in zip(historical.index, historical.values):
        ax.annotate(f'{int(value)}', xy=(year, value), xytext=(0, 3),
                    fontsize=8, ha='right', textcoords='offset points',
                    color='black', weight='bold')

In [28]:
def create_infection_and_parasite_summary_statistics_(results):
    """Create summary statistics table for diseases"""
    summary_data = []

    for disease, data in results.items():
        historical = data['historical']

        # Calculate statistics
        mean_val = historical.mean()
        std_val = historical.std()
        trend = (historical.iloc[-1] - historical.iloc[0]) / len(historical)

        # Average forecast across methods
        forecasts = []
        for method in ['arima', 'prophet', 'rf', 'linear']:
            try:
                method_forecast = data[method]['forecast']
                if hasattr(method_forecast, '__len__') and len(method_forecast) >= 3:
                    forecasts.append(method_forecast)
            except:
                continue

        if forecasts:
            avg_forecast_2024 = np.mean([f[0] for f in forecasts])
            avg_forecast_2025 = np.mean([f[1] for f in forecasts])
            avg_forecast_2026 = np.mean([f[2] for f in forecasts])
        else:
            # Fallback values if no forecasts available
            avg_forecast_2024 = historical.iloc[-1]
            avg_forecast_2025 = historical.iloc[-1]
            avg_forecast_2026 = historical.iloc[-1]

        summary_data.append({
            'Xəstəlik': disease,
            'Orta (2000-2023)': f"{mean_val:.0f}",
            'Standart sapma': f"{std_val:.0f}",
            'İllik trend': f"{trend:.1f}",
            '2024 proqnozu': f"{avg_forecast_2024:.0f}",
            '2025 proqnozu': f"{avg_forecast_2025:.0f}",
            '2026 proqnozu': f"{avg_forecast_2026:.0f}"
        })

    return pd.DataFrame(summary_data)

In [29]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime, timedelta
import numpy as np
from PyPDF2 import PdfMerger
import matplotlib.gridspec as gridspec # Import gridspec

def draw_infection_and_parasite_summary_figure(pdf, total_cases, diseases_2023, pie_labels, pie_sizes, growth_rates):
    """A4 səhifəsində 4 qrafiki estetik şəkildə çəkmək və PDF-ə əlavə etmək"""
    fig = plt.figure(figsize=(8.3, 11.7))  # A4 ölçüsü
    gs = gridspec.GridSpec(4, 1, height_ratios=[1, 1, 1, 1])
    fig.subplots_adjust(hspace=0.5)

    # Qrafik 1 – Ümumi Xəstəlik Hallarının Sayı
    ax1 = fig.add_subplot(gs[0])

    # Ensure we're working with valid numeric values
    valid_years = total_cases.index.year[pd.notna(total_cases.values)]
    valid_values = total_cases.values[pd.notna(total_cases.values)]

    ax1.plot(valid_years, valid_values, color='#1976D2', linewidth=2, marker='o', markersize=4)
    ax1.set_title('Ümumi İnfeksion Xəstəlik Halları (2000-2023)', fontsize=12, fontweight='bold', pad=8)
    ax1.set_xlabel('İl', fontsize=9)
    ax1.set_ylabel('Xəstəlik hallarının sayı', fontsize=9)
    ax1.grid(True, alpha=0.3)

    # Only annotate if we have valid data
    if len(valid_values) > 0:
        last_value = valid_values[-1]
        ax1.annotate(f"{int(last_value)}",
                     xy=(valid_years[-1], last_value),
                     xytext=(5, 0), textcoords='offset points',
                     fontsize=8, fontweight='bold', color='#1976D2')
    ax1.tick_params(axis='both', labelsize=8)

    # Chart 2 - Disease distribution (bar)
    ax2 = fig.add_subplot(gs[1])
    # Sort diseases by count for better visualization
    sorted_diseases = sorted(diseases_2023.items(), key=lambda item: item[1], reverse=True)
    bar_labels = [item[0] for item in sorted_diseases]
    bar_values = [item[1] for item in sorted_diseases]

    bars = ax2.barh(bar_labels, bar_values, color='#388E3C', alpha=0.85)
    ax2.set_title('2023-cü İldə Ən Çox Yayılmış İnfeksion Xəstəliklər',
                 fontsize=11, fontweight='bold', pad=6)
    ax2.set_xlabel('Xəstəlik hallarının sayı', fontsize=9)
    ax2.tick_params(axis='both', labelsize=8)
    ax2.set_xlim(0, max(bar_values) * 1.2)

    for label in ax2.get_yticklabels():
        label.set_horizontalalignment('right')

    for bar in bars:
        ax2.text(bar.get_width() + max(bar_values) * 0.02,
                 bar.get_y() + bar.get_height()/2,
                 f"{int(bar.get_width())}", va='center',
                 fontsize=7.5, fontweight='bold')

    # Chart 3 - Disease distribution (pie)
    ax3 = fig.add_subplot(gs[2])
    pie_colors = plt.cm.viridis(np.linspace(0, 1, len(pie_labels)))

    # Combine small slices into 'Others'
    threshold = sum(pie_sizes) * 0.03 # Diseases less than 3%
    small_slices = [size for size in pie_sizes if size < threshold]
    large_slices = [size for size in pie_sizes if size >= threshold]
    small_labels = [label for label, size in zip(pie_labels, pie_sizes) if size < threshold]
    large_labels = [label for label, size in zip(pie_labels, pie_sizes) if size >= threshold]

    if small_slices:
        large_slices.append(sum(small_slices))
        large_labels.append('Digərləri')

    ax3.pie(large_slices, labels=large_labels, autopct='%1.0f%%',
            startangle=140, colors=plt.cm.viridis(np.linspace(0, 1, len(large_labels))),
            textprops={'fontsize': 8}, pctdistance=0.75, radius=0.65)
    ax3.set_title('2023-cü İl üzrə İnfeksion Xəstəliklərin Paylanması',
                 fontsize=11, fontweight='bold', pad=6)


    # Chart 4 - Growth rates
    ax4 = fig.add_subplot(gs[3])
    growth_colors = ['#388E3C' if x > 0 else '#D32F2F' for x in growth_rates.values()]
    bars2 = ax4.bar(list(growth_rates.keys()), list(growth_rates.values()),
                    color=growth_colors, alpha=0.85)
    ax4.set_title('Seçilmiş Xəstəliklərdə Artım Dərəcəsi (%) (2000-2023)',
                 fontsize=12, fontweight='bold', pad=6)
    ax4.set_ylabel('Artım faizi', fontsize=9)
    ax4.tick_params(axis='x', labelsize=8, rotation=30)
    ax4.tick_params(axis='y', labelsize=8)

    for bar in bars2:
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                 f"{bar.get_height():.1f}%", ha='center', va='bottom' if bar.get_height() > 0 else 'top',
                 fontsize=7.5, fontweight='bold')
    ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()

In [30]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime
import numpy as np
from PyPDF2 import PdfMerger

In [31]:
def generate_infection_and_parasite_pdf_report(results):
    """Generate comprehensive PDF report for diseases"""
    A4_SIZE = (8.3, 11.7)
    generated_pdf_path = f"{field}_Hesabat_{end_year}.pdf"
    additional_pdf_path = "Qeydler.pdf"
    merged_pdf_path = "Birləşdirilmiş_Hesabat.pdf"

    # First create the title page separately
    title_pdf_path = f"Title_{field}_{end_year}.pdf"

    with PdfPages(title_pdf_path) as pdf:
        fig, ax = plt.subplots(figsize=A4_SIZE)
        ax.axis('off')
        # Dynamic title & author
        ax.text(0.5, 0.90, f'{field} Hesabat {end_year}',
                ha='center', fontsize=24, fontweight='bold')
        ax.text(0.5, 0.85,
                f'Zaman Seriyası Analizi və Maşın Öyrənməsi Proqnozları ({start_year}–{end_year})',
                ha='center', fontsize=16)
        ax.text(0.5, 0.80, f'Hazırladı: {name} {surname}',
                ha='center', fontsize=14)
        ax.text(0.5, 0.75,
                f'Tarix: {datetime.now().strftime("%d.%m.%Y")}',
                ha='center', fontsize=12)

        rect = patches.Rectangle((0.10, 0.65), 0.80, 0.05,
                                 linewidth=2, edgecolor='blue',
                                 facecolor='lightblue', alpha=0.3)
        ax.add_patch(rect)
        plt.tight_layout()
        pdf.savefig(fig)
        plt.close()

    # Create the main content pages
    content_pdf_path = f"Content_{field}_{end_year}.pdf"

    with PdfPages(content_pdf_path) as pdf:
        # Summary page with statistics
        # Calculate total cases for all diseases (sum of selected diseases)
        # Handle potential NaN values by filling with 0
        total_cases = pd.Series(0, index=next(iter(results.values()))['historical'].index)
        for data in results.values():
            total_cases += data['historical'].fillna(0)

        diseases_2023 = {}
        for disease, data in results.items():
            diseases_2023[disease] = data['historical'].iloc[-1]

        pie_labels = list(diseases_2023.keys())
        pie_sizes = list(diseases_2023.values())

        growth_rates = {}
        for disease, data in results.items():
            hist = data['historical']
            # Handle potential division by zero if the first value is 0 or NaN
            initial_value = hist.iloc[0] if pd.notna(hist.iloc[0]) and hist.iloc[0] != 0 else 1e-9
            growth_rate = ((hist.iloc[-1] - initial_value) / initial_value) * 100
            growth_rates[disease] = growth_rate


        fig, axs = plt.subplots(2, 2, figsize=(18, 12))
        fig.suptitle('STATİSTİK MƏLUMATLAR - VİZUAL İCMAL', fontsize=28, fontweight='bold', color='black', y=0.98)

        draw_infection_and_parasite_summary_figure(pdf, total_cases, diseases_2023, pie_labels, pie_sizes, growth_rates)

        # Individual disease forecasts
        for disease, data in results.items():
            fig = plt.figure(figsize=A4_SIZE)
            ax = fig.add_axes([0.1, 0.5, 0.87, 0.45])
            create_forecast_plot_2(disease, data, ax)
            plt.tight_layout()
            pdf.savefig(fig)
            plt.close()

    # Merge PDFs in the correct order: Title -> Notes -> Content
    merger = PdfMerger()
    merger.append(title_pdf_path)         # Title page first
    # merger.append(additional_pdf_path)    # Notes second (right after title)
    merger.append(content_pdf_path)       # Content pages last
    merger.write(merged_pdf_path)
    merger.close()

    # Clean up temporary files (optional)
    import os
    try:
        os.remove(title_pdf_path)
        os.remove(content_pdf_path)
    except:
        pass  # If files don't exist or can't be deleted, continue

print("PDF hesabatı yaradılır...")
results = perform_infection_and_parasite_analysis()
generate_infection_and_parasite_pdf_report(results)
print("Hesabat tamamlandı: 'Birləşdirilmiş_Hesabat.pdf'")

PDF hesabatı yaradılır...


INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeh_b71jg/ud0huqe9.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeh_b71jg/3m9szj7u.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=49473', 'data', 'file=/tmp/tmpeh_b71jg/ud0huqe9.json', 'init=/tmp/tmpeh_b71jg/3m9szj7u.json', 'output', 'file=/tmp/tmpeh_b71jg/prophet_modelyfbxc7o5/prophet_model-20250627165908.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
16:59:08 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:59:08 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:n_changepoints greater than number of observations. Using 5.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeh_b71jg/38o_yqda.json
DEBUG:cmdstanpy:input tempfile

Hesabat tamamlandı: 'Birləşdirilmiş_Hesabat.pdf'
